# Loading Libraries

In [38]:
import torch, pandas, scipy 
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn as nn
from csv import reader
import pandas as pd


# Pre procesamiento de datos.

In [39]:
# cargar archivo csv
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [40]:
# convertir los valores de string a float.
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [41]:
# convertir valores string a enteros.
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
            row[column] = lookup[row[column]]
    return lookup

**Lectura de datos del dataset**

In [42]:
#data = pd.read_csv('QCM10.csv',encoding = "ISO-8859-1")
#data.head()

In [43]:
#lectura de los datos del archivo csv 
xy = np.loadtxt('QCM10.csv', delimiter=";", dtype=np.float32, skiprows=1)
X = xy[:,0:14]
y = xy[:, [0]]

#se dividen los datos en entrada(X) y salida esperada(y)
X = torch.tensor(X, dtype = torch.float)
y = torch.tensor(y, dtype = torch.float)

#se genera una matriz de prediccion de los valores de entrada
xPredicted = torch.tensor(X, dtype = torch.float)

#se impriment las variables
print(X)
print(y)

tensor([[ -11.9800,  -10.9900,  -19.1200,  -17.2800,  -33.1300,  -28.4500,
          -48.8300,  -40.7700,  -62.4900,  -50.8200,    1.0000,    0.0000,
            0.0000,    0.0000],
        [ -12.1500,  -11.3300,  -22.3300,  -19.9500,  -39.8200,  -33.6400,
          -56.9000,  -46.7700,  -73.3200,  -58.9600,    1.0000,    0.0000,
            0.0000,    0.0000],
        [ -12.5800,  -11.7400,  -26.6700,  -23.3400,  -46.4800,  -38.6900,
          -65.9500,  -53.4600,  -84.5300,  -67.2100,    1.0000,    0.0000,
            0.0000,    0.0000],
        [ -13.7900,  -12.8200,  -30.5600,  -26.1800,  -52.3000,  -42.9800,
          -73.8100,  -59.1900,  -94.4100,  -74.4000,    1.0000,    0.0000,
            0.0000,    0.0000],
        [ -15.7300,  -13.8700,  -34.5400,  -28.6500,  -57.4400,  -46.2600,
          -80.3700,  -63.4900, -102.9400,  -80.2500,    1.0000,    0.0000,
            0.0000,    0.0000],
        [ -58.4100,  -38.6600,  -83.5800,  -57.3300, -110.2400,  -76.8000,
         -133.7

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


**Normalización de los valore de entrada y salida**

In [44]:
# calculo del valor maximo de X
X_max, _ = torch.max(X, 0)
# calculo del valor maximo de la entrada predecida
xPredicted_max, _ = torch.max(xPredicted, 0)
# normalizacion de los valores de entrada
X = torch.div(X, X_max)
# Normalizacion de los valores de entrada predecidos
xPredicted_max = torch.div(xPredicted, xPredicted_max)
# Normalizacion de los valores de salida
y = y / 100

print(xPredicted)
print("x  normalizado")
print(X)
print("----dimensiones de la entrada-----")
print(X.shape)

print("----dimensiones de la salida-----")
print(y.shape)


tensor([[ -11.9800,  -10.9900,  -19.1200,  -17.2800,  -33.1300,  -28.4500,
          -48.8300,  -40.7700,  -62.4900,  -50.8200,    1.0000,    0.0000,
            0.0000,    0.0000],
        [ -12.1500,  -11.3300,  -22.3300,  -19.9500,  -39.8200,  -33.6400,
          -56.9000,  -46.7700,  -73.3200,  -58.9600,    1.0000,    0.0000,
            0.0000,    0.0000],
        [ -12.5800,  -11.7400,  -26.6700,  -23.3400,  -46.4800,  -38.6900,
          -65.9500,  -53.4600,  -84.5300,  -67.2100,    1.0000,    0.0000,
            0.0000,    0.0000],
        [ -13.7900,  -12.8200,  -30.5600,  -26.1800,  -52.3000,  -42.9800,
          -73.8100,  -59.1900,  -94.4100,  -74.4000,    1.0000,    0.0000,
            0.0000,    0.0000],
        [ -15.7300,  -13.8700,  -34.5400,  -28.6500,  -57.4400,  -46.2600,
          -80.3700,  -63.4900, -102.9400,  -80.2500,    1.0000,    0.0000,
            0.0000,    0.0000],
        [ -58.4100,  -38.6600,  -83.5800,  -57.3300, -110.2400,  -76.8000,
         -133.7

# Modelo de red neuronal de retropropagación

In [45]:
class Neural_Network(nn.Module):
    def __init__(self, ):
        super(Neural_Network, self).__init__()
        # parametros(capas de la red).
        self.inputSize = 14
        self.outputSize = 1
        self.hiddenSize = 25
        
        # pesos
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) # tensor de 14 X 25
        self.W2 = torch.randn(self.hiddenSize, self.outputSize) # tensor de 25 X 1 
        print("------pesos w1 --------------")
        print(self.W1.shape)
        print("------pesos w1 --------------")
        print(self.W2.shape)

    #fordward propagation
    def forward(self, X):
        # la primera salida sera el resultado de la multiplicacion matricial entre la entrada y los pesos
        self.z = torch.matmul(X, self.W1) # 3 X 3 ".dot" does not broadcast in PyTorch
        # la salida sera alterada por la funcion de activacion sigmoidal
        self.z2 = self.sigmoid(self.z) # activation function
        # la tercera salida sera la multiplicacion matricial de la salida anterior por los nuevos pesos
        self.z3 = torch.matmul(self.z2, self.W2)
        # la ultima salida (el valor predecido) sera alterada por la funcion de activacion sigmoidal
        o = self.sigmoid(self.z3) 
        return o

    #calculo de la funcion sigmoidal de fordward propagation
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    #calculo de la derivada de la funcion sigmoidal, para la retropropagacion.
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    #retropropagacion
    def backward(self, X, y, o):
        # el error es la diferencia del valor obtenido y el esperado.
        self.o_error = y - o 
        # calculo de la influencia de la salida en el error al variar el peso.
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        # calculo de la influencia de la salida en el error al variar el peso.
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))

        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        #re ajuste de los ultimos pesos
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
        #re ajuste de los pesos anteriores
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)

        
    # entrenamiendo de la red neuronal    
    def train(self, X, y):
        # fordward propagation
        o = self.forward(X)
        # backpropagation
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # guardado de los pesos con la funcion save de torch
        torch.save(model, "bpnn_saved")
        # el metodo de carga de pesos sera load(ruta_archivo)
        
    # prediccion de los valores, dado una matriz de mismas dimensionalidades.    
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))

In [46]:
NN = Neural_Network()
for i in range(500):  # entrenar la red 500 veces
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
NN.saveWeights(NN)
NN.predict()

------pesos w1 --------------
torch.Size([14, 25])
------pesos w1 --------------
torch.Size([25, 1])
#0 Loss: 1.9119738340377808
#1 Loss: 0.24852041900157928
#2 Loss: 0.24852041900157928
#3 Loss: 0.24852041900157928
#4 Loss: 0.24852041900157928
#5 Loss: 0.24852041900157928
#6 Loss: 0.24852041900157928
#7 Loss: 0.24852041900157928
#8 Loss: 0.24852041900157928
#9 Loss: 0.24852041900157928
#10 Loss: 0.24852041900157928
#11 Loss: 0.24852041900157928
#12 Loss: 0.24852041900157928
#13 Loss: 0.24852041900157928
#14 Loss: 0.24852041900157928
#15 Loss: 0.24852041900157928
#16 Loss: 0.24852041900157928
#17 Loss: 0.24852041900157928
#18 Loss: 0.24852041900157928
#19 Loss: 0.24852041900157928
#20 Loss: 0.24852041900157928
#21 Loss: 0.24852041900157928
#22 Loss: 0.24852041900157928
#23 Loss: 0.24852041900157928
#24 Loss: 0.24852041900157928
#25 Loss: 0.24852041900157928
#26 Loss: 0.24852041900157928
#27 Loss: 0.24852041900157928
#28 Loss: 0.24852041900157928
#29 Loss: 0.24852041900157928
#30 Loss: 